# Filings Daily Stats

We need to load in these libraries into our notebook in order to query, load, manipulate and view the data

In [ ]:
import os
import csv
from datetime import datetime

%load_ext sql
%load_ext dotenv
%config SqlMagic.displaylimit = 5

In [ ]:
from dotenv import find_dotenv
from dotenv import load_dotenv

load_dotenv(find_dotenv())

In [ ]:
%dotenv -o

### DB Connection

In [ ]:
import google.auth
from google.cloud.sql.connector import Connector
import sqlalchemy

DB_USER = os.getenv("DATABASE_USERNAME", "")
DB_NAME = os.getenv("DATABASE_NAME", "")
DB_PASSWORD = os.getenv("DATABASE_PASSWORD", "")
DB_HOST = os.getenv("DATABASE_HOST", "")
DB_PORT = os.getenv("DATABASE_PORT", "")
DB_CONNECTION_NAME = os.getenv("LEAR_DB_CONNECTION_NAME")  # project:region:instance-name

# initialize Connector object
connector = Connector()

# function to return the database connection object
def get_conn():
    conn = connector.connect(
        DB_CONNECTION_NAME,
        "pg8000",
        ip_type="public",
        user=DB_USER,
        db=DB_NAME,
        enable_iam_auth=True
    )
    return conn

def get_pool():
    # create connection pool with 'creator' argument to our connection object function
    if not DB_CONNECTION_NAME:
        connect_string = f'postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}'
        pool = sqlalchemy.create_engine(connect_string)
    else:
        # NOTE: also need GOOGLE_APPLICATION_CREDENTIALS to be set in the .env
        #    - when running the notebook on its own, the ^ cred path starts in this folder instead of the root of the service
        #    - if this path is invalid or the creds are invalid the line below will fail
        credentials, project_id = google.auth.default()
        pool = sqlalchemy.create_engine(
            "postgresql+pg8000://",
            creator=get_conn,
        )
    
    return pool

In [ ]:
lear_pool = get_pool()
%sql lear_pool --alias lear
%sql lear

In [ ]:
%sql --connections

Simple query to run to ensure our libraries are loaded and our DB connection is working

In [ ]:
%%sql 
select now() AT TIME ZONE 'PST' as current_date

### Update db

In [ ]:
%%sql 
update sent_to_gazette
set sent_to_gazette_date = now() 
from (select f.id from  filings f, businesses b, sent_to_gazette stg, businesses_version old, businesses_version new  
          where f.filing_type in ('alteration', 'changeOfName')                  
                and f.business_id=b.id
                and f.id=stg.filing_id
                and b.legal_type in ('CP', 'XCP')
                and f.transaction_id=old.end_transaction_id 
                and f.transaction_id=new.transaction_id                
                and old.legal_name != new.legal_name
                and stg.sent_to_gazette_date is null 
    ) AS subquery
where sent_to_gazette.filing_id=subquery.id; 

In [ ]:
%%sql 
update sent_to_gazette
set sent_to_gazette_date = now() 
from (select f.id from  filings f, businesses b, sent_to_gazette stg  
          where f.filing_type in ('dissolved', 'voluntaryLiquidation', 'dissolution') 
                and f.business_id=b.id
                and f.id=stg.filing_id
                and b.legal_type in ('CP', 'XCP')
                and stg.sent_to_gazette_date is null
    ) AS subquery
where sent_to_gazette.filing_id=subquery.id; 

In [ ]:
%%sql 
update sent_to_gazette
set sent_to_gazette_date = now() 
from (select f.id from  filings f, businesses b, sent_to_gazette stg    
          where f.filing_type in ('incorporationApplication','amalgamationApplication')      
                and f.business_id=b.id
                and f.id=stg.filing_id
                and b.legal_type in ('CP', 'XCP')
                and stg.sent_to_gazette_date is null
    ) AS subquery
where sent_to_gazette.filing_id=subquery.id; 

In [ ]:
%%sql 
update sent_to_gazette
set sent_to_gazette_date = now() 
from (select f.id from  filings f, businesses b, sent_to_gazette stg    
          where f.filing_type = 'restoration'    
                and f.business_id=b.id
                and f.id=stg.filing_id
                and b.legal_type in ('CP', 'XCP')
                and stg.sent_to_gazette_date is null
    ) AS subquery
where sent_to_gazette.filing_id=subquery.id; 